** GANs for generating synthetic datasets**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Generator
class Generator(nn.Module):
    def __init__(self, noise_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(noise_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim),
            nn.Tanh()  # Output range [-1, 1]
        )

    def forward(self, x):
        return self.model(x)

# Discriminator
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()  # Probability output
        )

    def forward(self, x):
        return self.model(x)

# Hyperparameters
noise_dim = 100
data_dim = 784  # e.g., flattened 28x28 images
lr = 0.0002
epochs = 200
batch_size = 64

# Initialize models and optimizers
G = Generator(noise_dim, data_dim)
D = Discriminator(data_dim)
G_optimizer = optim.Adam(G.parameters(), lr=lr)
D_optimizer = optim.Adam(D.parameters(), lr=lr)

# Loss function
criterion = nn.BCELoss()

# Training loop (simplified)
for epoch in range(epochs):
    # Assume real_data is a batch from the dataset
    real_data = torch.randn(batch_size, data_dim)  # Placeholder
    real_labels = torch.ones(batch_size, 1)
    fake_labels = torch.zeros(batch_size, 1)

    # Train Discriminator
    D_optimizer.zero_grad()
    real_output = D(real_data)
    d_loss_real = criterion(real_output, real_labels)

    noise = torch.randn(batch_size, noise_dim)
    fake_data = G(noise)
    fake_output = D(fake_data.detach())
    d_loss_fake = criterion(fake_output, fake_labels)

    d_loss = d_loss_real + d_loss_fake
    d_loss.backward()
    D_optimizer.step()

    # Train Generator
    G_optimizer.zero_grad()
    fake_output = D(fake_data)
    g_loss = criterion(fake_output, real_labels)  # Fool the Discriminator
    g_loss.backward()
    G_optimizer.step()

    print(f"Epoch {epoch}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

# Generate synthetic data
noise = torch.randn(10, noise_dim)
synthetic_data = G(noise)

Epoch 0, D Loss: 1.3849598169326782, G Loss: 0.6848213076591492
Epoch 1, D Loss: 1.3269908428192139, G Loss: 0.6908121109008789
Epoch 2, D Loss: 1.3303923606872559, G Loss: 0.6952741742134094
Epoch 3, D Loss: 1.3170936107635498, G Loss: 0.7015909552574158
Epoch 4, D Loss: 1.3430728912353516, G Loss: 0.7062100172042847
Epoch 5, D Loss: 1.3042454719543457, G Loss: 0.7106066942214966
Epoch 6, D Loss: 1.3027377128601074, G Loss: 0.7131630182266235
Epoch 7, D Loss: 1.3012773990631104, G Loss: 0.7199525833129883
Epoch 8, D Loss: 1.2798326015472412, G Loss: 0.7231944799423218
Epoch 9, D Loss: 1.253566026687622, G Loss: 0.7298039197921753
Epoch 10, D Loss: 1.2780611515045166, G Loss: 0.7312931418418884
Epoch 11, D Loss: 1.2739906311035156, G Loss: 0.7366844415664673
Epoch 12, D Loss: 1.2737189531326294, G Loss: 0.7363312244415283
Epoch 13, D Loss: 1.2524734735488892, G Loss: 0.7412818074226379
Epoch 14, D Loss: 1.2437070608139038, G Loss: 0.7450071573257446
Epoch 15, D Loss: 1.2442865371704102